# Autograd
- provides automatic differentiation for tensor operations. It enables gradient computation, which is essential for training machine learning models using optimization algorithms like gradient descent.

In [1]:
import torch

## we have:
- y = x^2
- we find dy/dx at x = 3

In [4]:
x = torch.tensor(3.0, requires_grad=True)
# if we need the derivative of any tensor, while defining it we use requires_grad = True, by default it is False


In [5]:
y = x**2

In [6]:
x

tensor(3., requires_grad=True)

In [7]:
y

tensor(9., grad_fn=<PowBackward0>)

tensor(9., grad_fn=PowBackward0)

this remembers which operation was performed so that it can be retraced

In [8]:
## calculating the derivative

y.backward()

In [11]:
## checking the result
x.grad

tensor(6.)

### we now have a nested function:
- y = x^2
- z = sin(y)
- we calculate dz/dx
- let us take x = 3

In [12]:
x = torch.tensor(3.0, requires_grad=True)
y = x**2
z = torch.sin(y)

In [13]:
print(x)
print(y)
print(z)

tensor(3., requires_grad=True)
tensor(9., grad_fn=<PowBackward0>)
tensor(0.4121, grad_fn=<SinBackward0>)


In [14]:
z.backward()

In [15]:
x.grad

tensor(-5.4668)

- a computation graph is constructed.
- we can't calculate the gradient of intermediate gradients


### A small neural network

In [16]:
x = torch.tensor(6.7) # input feature
y = torch.tensor(0.0) # True label

w = torch.tensor(1.0) # weight
b = torch.tensor(0.0) # bias

In [17]:
# Binary Cross-Entropy Loss for scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [18]:
# Forward pass
z = w * x + b  # Weighted sum (linear part)
y_pred = torch.sigmoid(z)  # Predicted probability

# Compute binary cross-entropy loss
loss = binary_cross_entropy_loss(y_pred, y)

In [19]:
loss

tensor(6.7012)

In [20]:
# Derivatives:
# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [21]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


### Using Autograd

In [22]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [23]:
z = w*x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [24]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [25]:
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [26]:
loss.backward()

In [28]:
print(f"Gradient of loss w.r.t weight using Autograd: {w.grad}")
print(f"Gradient of loss w.r.t bias (db) using Autograd: {b.grad}")

Gradient of loss w.r.t weight using Autograd: 6.6917619705200195
Gradient of loss w.r.t bias (db) using Autograd: 0.9987704753875732


## Using Vectors


In [29]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

In [30]:
x

tensor([1., 2., 3.], requires_grad=True)

In [31]:
y = (x**2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [32]:
y.backward()

In [33]:
x.grad

tensor([0.6667, 1.3333, 2.0000])

- if we run backward pass multiple times, then gradients keeps accumulating

In [34]:
## clearing grad

x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [42]:
y = x**2
y

tensor(4., grad_fn=<PowBackward0>)

In [43]:
y.backward()

In [44]:
x.grad

tensor(4.)

- ideally it should be 4, but on running twice it becomes 8 which is not desired

In [41]:
x.grad.zero_()

tensor(0.)

## Disable gradient tracking

In [45]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [46]:
y = x**2
y

tensor(4., grad_fn=<PowBackward0>)

In [47]:
y.backward()

In [48]:
x.grad

tensor(4.)

- # we disable the gradient tracking
### option 1 - requires_grad_(False)
### option 2 - detach()
### option 3 - torch.no_grad()

In [49]:
x.requires_grad_(False)

tensor(2.)

In [50]:
x

tensor(2.)

In [51]:
y = x ** 2

In [52]:
y

tensor(4.)

In [53]:
y.backward()

## won't run

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [54]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [55]:
z = x.detach()
z

tensor(2.)

In [56]:
y = x ** 2

In [57]:
y1 = z ** 2
y1

tensor(4.)

In [58]:
y.backward()

In [59]:
x.grad

tensor(4.)

In [60]:
y1.backward()
## this won't run

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [61]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [62]:
with torch.no_grad():
  y = x**2

In [63]:
y

tensor(4.)

In [64]:
y.backward()
## won't work

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn